In [34]:
%ls

convin_stashfin.pkl  generality_predictions/  lm_predictions/


In [2]:
%cd divyaksh

/DATA1/divyaksh


/home/ritesh/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [35]:
%ls

convin_stashfin.pkl  generality_predictions/  lm_predictions/


In [36]:
import pandas as pd

In [37]:
df = pd.read_pickle('convin_stashfin.pkl')

In [38]:
df.columns

Index(['domain', 'utterance_file', 'domain_batch', 'context_file', 'emotion',
       'fine-grained-emotion', 'text_embedding', 'audio_embedding',
       'machine_transcript', 'roman_transcript', 'native_transcript'],
      dtype='object')

In [39]:
df.shape

(260, 11)

In [40]:
df.dtypes

domain                  object
utterance_file          object
domain_batch            object
context_file            object
emotion                 object
fine-grained-emotion    object
text_embedding          object
audio_embedding         object
machine_transcript      object
roman_transcript        object
native_transcript       object
dtype: object

In [41]:
df['emotion'].value_counts()

emotion
Negative    134
Neutral     103
Junk         12
Positive     11
Name: count, dtype: int64

In [42]:
list = [i%2 for i in range(260)]
df["Speaker"] = pd.DataFrame(list)

In [43]:
df['Speaker']

0      0
1      1
2      0
3      1
4      0
      ..
255    1
256    0
257    1
258    0
259    1
Name: Speaker, Length: 260, dtype: int64

In [44]:
df_new = df.drop([i for i,x in enumerate(df['emotion']) if x == "Junk"])

In [45]:
df_new = df_new.reset_index()

In [46]:
df_new.shape

(248, 13)

In [47]:
df_new['emotion'].isnull().sum()

0

In [48]:
df_new['fine-grained-emotion'].value_counts()

fine-grained-emotion
Junk                 132
None of the above     74
Curiosity             40
Confusion              2
Name: count, dtype: int64

In [49]:
# df_new = df.drop(i for i,x in enumerate(df['fine-grained-emotion']) if x == "Junk" or x == "None of the above")

In [50]:
df_new.shape

(248, 13)

In [51]:
df_new.columns

Index(['index', 'domain', 'utterance_file', 'domain_batch', 'context_file',
       'emotion', 'fine-grained-emotion', 'text_embedding', 'audio_embedding',
       'machine_transcript', 'roman_transcript', 'native_transcript',
       'Speaker'],
      dtype='object')

In [52]:
df_new['emotion'].value_counts()

emotion
Negative    134
Neutral     103
Positive     11
Name: count, dtype: int64

In [53]:
df_final = df_new.drop(["domain", "utterance_file", "domain_batch", "context_file", "fine-grained-emotion", "machine_transcript", "native_transcript"], axis=1)

In [54]:
df_final.head(5)

,index,emotion,text_embedding,audio_embedding,roman_transcript,Speaker
0,1,Neutral,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[4.3364997, -0.15071316, 2.3584733, 1.979610...",hello hello good afternoon sir sube khan bat k...,1
1,2,Neutral,"[[[-0.515, 0.3608, -2.084, 1.298, 0.07574, -0....","[[[2.291522, 1.0063372, 0.8391347, 1.0891207, ...",ha ha sir main stashfin se khushi bat kar rahi...,0
2,3,Negative,"[[[-0.515, 0.3608, -2.084, 1.298, 0.07574, -0....","[[[0.7635638, 1.1420541, 0.801178, 0.21001612,...",payment nahi kar raha hu matalab matalab apane...,1
3,4,Negative,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[2.1685843, 0.4390855, 1.3647568, 1.7726477,...",mera kitane ka payment hua hai wait abhi batat...,0
4,5,Negative,"[[[-0.5146, 0.3616, -2.084, 1.297, 0.07574, -0...","[[[2.026393, 0.19344328, 3.0774066, -1.533694,...",apaka seven thousand ka payment hua hai to fir...,1


In [55]:
df_final["Label"] = pd.DataFrame([i for i in range(len(df_final))])

In [56]:
max_length = -float('inf')
min_length = float('inf')

# Iterate through each embedding in the 'text_embedding' column
for embedding in df['text_embedding']:
    for sublist in embedding:
        current_length = len(sublist)
        if current_length > max_length:
            max_length = current_length
        if current_length < min_length:
            min_length = current_length

# Display the max and min lengths
print("Max length in 2nd dimension:", max_length)
print("Min length in 2nd dimension:", min_length)

Max length in 2nd dimension: 42
Min length in 2nd dimension: 1


In [57]:
import numpy as np
def compute_average(array):
    return np.mean(array)

# Apply the function to each element in the 'text_emb' column and store the result in a new column
avg_text= df_final['text_embedding'].apply(compute_average)
avg_audio=df_final['audio_embedding'].apply(compute_average)

In [58]:
avg_audio

0      0.631411
1      0.651500
2      0.674491
3      0.612439
4      0.667051
         ...   
243    0.586290
244    0.631476
245    0.605755
246    0.616531
247    0.579473
Name: audio_embedding, Length: 248, dtype: float32

In [59]:
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [60]:
#converting to a 100 dim vector
import torch
import torch.nn as nn

transforming_layer =nn.Linear(1, 100)
compressed_text = [transforming_layer(torch.FloatTensor([x])).detach().numpy() for x in avg_text]
compressed_audio = [transforming_layer(torch.FloatTensor([x])).detach().numpy() for x in avg_text]

In [61]:
compressed_text[0]

array([-0.9431794 , -0.97845626, -0.74128795,  0.74492544, -0.755263  ,
        0.05561516, -0.8917958 ,  0.87965846, -0.6174083 ,  0.32230344,
       -0.24459414, -0.31823906, -0.33457482,  0.32867694, -0.10445685,
       -0.5057833 , -0.0588034 , -0.19312546,  0.6342764 , -0.32870844,
        0.463033  ,  0.9046372 ,  0.894857  , -0.5630702 ,  0.57969725,
       -0.4179213 , -0.42786652, -0.7100137 ,  0.07504687, -0.2674076 ,
       -0.9116996 ,  0.20114543,  0.97124   , -0.01862704,  0.24647519,
        0.7966276 , -0.82050043,  0.26501608, -0.97287166,  0.06941102,
       -0.23764227,  0.51632196,  0.51280576, -0.65430886, -0.2962073 ,
        0.37955928, -0.46737313,  0.24510194, -0.12558901, -0.33672956,
        0.76508296,  0.08959527,  0.8742131 ,  0.29882574,  0.31683198,
        0.73171836,  0.3136283 , -0.01090018, -0.30817205, -0.42844367,
        0.2766638 ,  0.39414978, -0.93704784,  0.729875  , -0.88094383,
       -0.44514808, -0.7433097 ,  0.6816421 ,  0.38294187,  0.61

In [62]:
df_final.head(5)

,index,emotion,text_embedding,audio_embedding,roman_transcript,Speaker,Label
0,1,Neutral,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[4.3364997, -0.15071316, 2.3584733, 1.979610...",hello hello good afternoon sir sube khan bat k...,1,0
1,2,Neutral,"[[[-0.515, 0.3608, -2.084, 1.298, 0.07574, -0....","[[[2.291522, 1.0063372, 0.8391347, 1.0891207, ...",ha ha sir main stashfin se khushi bat kar rahi...,0,1
2,3,Negative,"[[[-0.515, 0.3608, -2.084, 1.298, 0.07574, -0....","[[[0.7635638, 1.1420541, 0.801178, 0.21001612,...",payment nahi kar raha hu matalab matalab apane...,1,2
3,4,Negative,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[2.1685843, 0.4390855, 1.3647568, 1.7726477,...",mera kitane ka payment hua hai wait abhi batat...,0,3
4,5,Negative,"[[[-0.5146, 0.3616, -2.084, 1.297, 0.07574, -0...","[[[2.026393, 0.19344328, 3.0774066, -1.533694,...",apaka seven thousand ka payment hua hai to fir...,1,4


In [63]:
df_final = df_final.drop('index', axis=1)

In [64]:
df_final.head(5)

,emotion,text_embedding,audio_embedding,roman_transcript,Speaker,Label
0,Neutral,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[4.3364997, -0.15071316, 2.3584733, 1.979610...",hello hello good afternoon sir sube khan bat k...,1,0
1,Neutral,"[[[-0.515, 0.3608, -2.084, 1.298, 0.07574, -0....","[[[2.291522, 1.0063372, 0.8391347, 1.0891207, ...",ha ha sir main stashfin se khushi bat kar rahi...,0,1
2,Negative,"[[[-0.515, 0.3608, -2.084, 1.298, 0.07574, -0....","[[[0.7635638, 1.1420541, 0.801178, 0.21001612,...",payment nahi kar raha hu matalab matalab apane...,1,2
3,Negative,"[[[-0.5146, 0.3608, -2.084, 1.298, 0.07574, -0...","[[[2.1685843, 0.4390855, 1.3647568, 1.7726477,...",mera kitane ka payment hua hai wait abhi batat...,0,3
4,Negative,"[[[-0.5146, 0.3616, -2.084, 1.297, 0.07574, -0...","[[[2.026393, 0.19344328, 3.0774066, -1.533694,...",apaka seven thousand ka payment hua hai to fir...,1,4


In [65]:
df_final = df_final.drop(['text_embedding', 'audio_embedding'], axis=1)

In [66]:
df_final['text_embedding']=compressed_text
df_final['audio_embedding']=compressed_audio

In [67]:
df_final.head(5)

,emotion,roman_transcript,Speaker,Label,text_embedding,audio_embedding
0,Neutral,hello hello good afternoon sir sube khan bat k...,1,0,"[-0.9431794, -0.97845626, -0.74128795, 0.74492...","[-0.9431794, -0.97845626, -0.74128795, 0.74492..."
1,Neutral,ha ha sir main stashfin se khushi bat kar rahi...,0,1,"[-0.94331944, -0.97864723, -0.74149364, 0.7446...","[-0.94331944, -0.97864723, -0.74149364, 0.7446..."
2,Negative,payment nahi kar raha hu matalab matalab apane...,1,2,"[-0.9412185, -0.97578174, -0.7384069, 0.748762...","[-0.9412185, -0.97578174, -0.7384069, 0.748762..."
3,Negative,mera kitane ka payment hua hai wait abhi batat...,0,3,"[-0.9403989, -0.9746639, -0.73720264, 0.750366...","[-0.9403989, -0.9746639, -0.73720264, 0.750366..."
4,Negative,apaka seven thousand ka payment hua hai to fir...,1,4,"[-0.9387058, -0.9723548, -0.7347152, 0.7536785...","[-0.9387058, -0.9723548, -0.7347152, 0.7536785..."


In [68]:
for i,val in enumerate(df_final['emotion']):
    if val=='Neutral':
        df_final.loc[i,'emotion'] = '0'
    elif val=='Positive':
        df_final.loc[i,'emotion'] = '1'
    else:
        df_final.loc[i,'emotion'] = '2'

In [69]:
df_final.head(10)

,emotion,roman_transcript,Speaker,Label,text_embedding,audio_embedding
0,0,hello hello good afternoon sir sube khan bat k...,1,0,"[-0.9431794, -0.97845626, -0.74128795, 0.74492...","[-0.9431794, -0.97845626, -0.74128795, 0.74492..."
1,0,ha ha sir main stashfin se khushi bat kar rahi...,0,1,"[-0.94331944, -0.97864723, -0.74149364, 0.7446...","[-0.94331944, -0.97864723, -0.74149364, 0.7446..."
2,2,payment nahi kar raha hu matalab matalab apane...,1,2,"[-0.9412185, -0.97578174, -0.7384069, 0.748762...","[-0.9412185, -0.97578174, -0.7384069, 0.748762..."
3,2,mera kitane ka payment hua hai wait abhi batat...,0,3,"[-0.9403989, -0.9746639, -0.73720264, 0.750366...","[-0.9403989, -0.9746639, -0.73720264, 0.750366..."
4,2,apaka seven thousand ka payment hua hai to fir...,1,4,"[-0.9387058, -0.9723548, -0.7347152, 0.7536785...","[-0.9387058, -0.9723548, -0.7347152, 0.7536785..."
5,2,sat hajar ka ye apaka case legal hai thats why...,0,5,"[-0.9353056, -0.9677173, -0.7297196, 0.7603314...","[-0.9353056, -0.9677173, -0.7297196, 0.7603314..."
6,0,to ye reference call kaun kar raha hai ye sirf...,1,6,"[-0.9337055, -0.965535, -0.7273687, 0.76346225...","[-0.9337055, -0.965535, -0.7273687, 0.76346225..."
7,0,company unhe call karati hamari taraf se nahin...,0,7,"[-0.93771243, -0.97099996, -0.73325574, 0.7556...","[-0.93771243, -0.97099996, -0.73325574, 0.7556..."
8,2,ki mujhe ye samajh men nahin aa raha hai ki ag...,1,8,"[-0.94430286, -0.97998846, -0.74293846, 0.7427...","[-0.94430286, -0.97998846, -0.74293846, 0.7427..."
9,2,do hajar kya ligal team men khali hai kya no n...,0,9,"[-0.93514895, -0.9675036, -0.7294894, 0.760638...","[-0.93514895, -0.9675036, -0.7294894, 0.760638..."


In [70]:
df_final.to_pickle('../stashfin_processed_new1.pkl') 